In [8]:

import sys
sys.path.append("/Users/utilizador/Documents/GitHub/si/src")
import numpy as np
from si.data.dataset import Dataset
from si.metrics.accuracy import accuracy
import pandas as pd
from si.data.dataset import Dataset
from si.model_selection.split import train_test_split
from si.models.logistic_regression import LogisticRegression
from si.models.decision_tree_classifier import DecisionTreeClassifier
from si.models.knn_classifier import KNNClassifier
from si.metrics.accuracy import accuracy
from si.io.csv_file import read_csv

A classe StackingClassifier implementa um método de aprendizado de máquina chamado Stacking. O Stacking é uma técnica de ensemble que combina previsões de múltiplos modelos base (chamados de modelos de primeiro nível) com um modelo final (chamado de meta-modelo) que aprende a partir dessas previsões para fazer a predição final.

__Stacking:__
Uma abordagem de ensemble que utiliza múltiplos modelos base para fazer previsões iniciais.
Um modelo final aprende a partir dessas previsões (como entradas) para gerar as predições finais.

__Estrutura do Stacking:__

Modelos Base: Modelos de primeiro nível que fazem predições iniciais. Essas predições são usadas como entrada para o modelo final.

Modelo Final: Um modelo que utiliza as predições dos modelos base para aprender as relações entre essas predições e o valor verdadeiro.

__Vantagens:__
Combina diferentes modelos para capturar a variabilidade nos dados.
Pode melhorar a generalização ao combinar modelos com diferentes pontos fortes.

__Desvantagens:__
Aumenta a complexidade computacional.
O modelo final pode ser sensível a erros sistemáticos dos modelos base.

__init__:
Inicializa o modelo com uma lista de modelos base e um modelo final.

__fit:__
Treina os modelos base no dataset original.
Usa as predições dos modelos base para criar um novo dataset (meta-features).
Treina o modelo final com esse novo dataset.

___predict:__
Faz predições com os modelos base no dataset fornecido.
Usa essas predições como entrada para o modelo final para gerar as predições finais.

___score:__
Calcula a acurácia do modelo ao comparar as predições finais com os rótulos verdadeiros.

In [7]:
class StackingClassifier:
    def __init__(self, models, final_model):
        """
        Initializes the StackingClassifier with an initial set of models and a final model.

        Parameters:
        - models: list of Model
            The initial set of models to generate predictions.
        - final_model: Model
            The final model that will make the final predictions based on the outputs of the initial models.
        """
        self.models = models
        self.final_model = final_model

    def _fit(self, dataset: Dataset):
        """
        Trains the ensemble models and the final model.

        Parameters:
        - dataset: Dataset
            The dataset containing the features (X) and labels (y).

        Returns:
        - self: StackingClassifier
            The trained StackingClassifier.
        """
      
        predictions = []
        for model in self.models:
            model._fit(dataset) # Ajusta o modelo base com o dataset
            predictions.append(model._predict(dataset))
        
        # Combina as predições dos modelos base em um novo conjunto de dados
        predictions = np.column_stack(predictions)
        final_dataset = Dataset(predictions, dataset.y)
        
        # Train the final model
        self.final_model._fit(final_dataset)
        
        return self

    def _predict(self, dataset: Dataset) -> np.ndarray:
        """
        Predicts the labels using the ensemble models and the final model.

        Parameters:
        - dataset: Dataset
            The dataset containing the features (X).

        Returns:
        - predictions: np.ndarray
            The predicted labels.
        """
        
        predictions = []
        for model in self.models:
            predictions.append(model._predict(dataset))
        
         # Combina as predições dos modelos base em um novo conjunto de dados
        predictions = np.column_stack(predictions)
        
        
        final_dataset = Dataset(predictions, None)  # Faz predições finais usando o modelo final
        return self.final_model._predict(final_dataset)

    def _score(self, dataset: Dataset) -> float:
        """
        Computes the accuracy between predicted and real labels.

        Parameters:
        - dataset: Dataset
            The dataset containing the features (X) and true labels (y).

        Returns:
        - accuracy: float
            The accuracy score of the model.
        """
        predictions = self._predict(dataset)
        return accuracy(dataset.y, predictions)


In [9]:
Path= "/Users/utilizador/Documents/GitHub/si/datasets/breast_bin/"
data = read_csv(Path + "breast-bin.csv", sep=",", label=True)
data.summary()

,feat_0,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8
mean,4.412607,3.133238,3.206304,2.809456,3.217765,3.478510,3.438395,2.866762,1.590258
median,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
var,7.909984,9.310328,8.831364,8.148507,4.901002,13.074753,5.945345,9.324655,2.940994


In [10]:

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)


In [11]:
# Base models
knn1 = KNNClassifier(k=3)                        
logistic = LogisticRegression()        
decision_tree = DecisionTreeClassifier(max_depth=5) 

# Final model
knn2 = KNNClassifier(k=5)  


In [12]:
# Stacking Classifier
stacking_model = StackingClassifier(models=[knn1, logistic, decision_tree], final_model=knn2)

# Train the StackingClassifier
stacking_model._fit(train_data)


In [13]:
# Compute the accuracy on the test set
stacking_accuracy = stacking_model._score(test_data)
print(f"StackingClassifier Accuracy: {stacking_accuracy}")


StackingClassifier Accuracy: 0.6220095693779905


In [ ]:
class TestStackingClassifier(TestCase):
    def setUp(self):
        # Create a synthetic dataset
        self.X, self.y = make_classification(n_samples=100, n_features=5, random_state=42)
        # Define base models and final model
        self.models = [DecisionTreeClassifier(max_depth=3, random_state=42) for _ in range(3)]
        self.final_model = LogisticRegression()
        self.clf = StackingClassifier(models=self.models, final_model=self.final_model)

    def test_fit(self):
        # Test if fit method works without errors
        self.clf.fit(self.X, self.y)
        for model in self.models:
            self.assertTrue(hasattr(model, "tree_"))  # Check if DecisionTreeClassifier is fitted

    def test_predict(self):
        # Test predict method
        self.clf.fit(self.X, self.y)
        predictions = self.clf.predict(self.X)
        self.assertEqual(len(predictions), len(self.y))

    def test_score(self):
        # Test score method
        self.clf.fit(self.X, self.y)
        score = self.clf.score(self.X, self.y)
        self.assertTrue(0 <= score <= 1)  # Accuracy must be between 0 and 1